In [ ]:
import os

os.chdir(r'/Users/jialunlyu/Documents/GitHub/ProjectBuffalo')

In [ ]:
import buffalo.ingestion as ingestion

In [ ]:
ingestion.configuration.Configuration.print()

In [ ]:
advantage_endpoint = ingestion.advantage.AdvantageStockGrepper()

In [ ]:
advantage_endpoint.stock_download(
    symbol='AAPL',
    interval='daily',
    year_slice=None, ## Intraday 
    adjusted=True
)

In [ ]:
advantage_endpoint.company_info_download(
    'CASH_FLOW',
    'IBM',
    '3month'
)

In [ ]:
advantage_endpoint.listing_info_download(date='2022-02-24')

In [ ]:
advantage_endpoint.trend_indicator_download(
    function='SMA',
    symbol='IBM',
    interval='1min',
    time_period=10,
    series_type='close'
)

In [ ]:
advantage_endpoint.trend_indicator_download(
    function='T3',
    symbol='IBM',
    interval='1min',
    time_period=10,
    series_type='close'
)

In [ ]:
advantage_endpoint.econ_download(
    function='FEDERAL_FUNDS_RATE',
    interval='daily'
)

In [ ]:
advantage_endpoint.econ_download(
    function='TREASURY_YIELD',
    interval='daily',
    maturity='3month'
)

In [ ]:
advantage_endpoint.econ_download(
    function='NONFARM_PAYROLL',
    interval='monthly'
)

In [ ]:
advantage_endpoint.econ_download(
    function='CPI',
    interval='monthly'
)

In [ ]:
advantage_endpoint.econ_download(
    function='UNEMPLOYMENT',
    interval='monthly'
)

In [ ]:
advantage_endpoint.cycle_indicator_download(
    function='ULTOSC',
    symbol='IBM',
    interval='1min',
    time_period=10,
    series_type='close'
)

In [ ]:
advantage_endpoint.cycle_indicator_download(
    function='ULTOSC',
    symbol='IBM',
    interval='1min',
    time_period=10,
    series_type='close'
)

In [ ]:
advantage_endpoint.market_news_sentiment_download('IBM', sort='Latest', limit=50)

In [ ]:
# import sqlite3


# fp = r'~/newsqlitefile.sqlite'

# import pandas as pd

# a = pd.DataFrame({"a": range(10), "v": range(1, 11)})

# newconn, _ = sqlite3.connect(fp)
# a.to_sql("a", newconn)

# newconn.close()